In [1]:
# https://www.kaggle.com/c/sf02classification

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import randint as randint
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
# загражаем данные
df_train = pd.read_csv('train.csv', header=0, index_col='_id', encoding='utf-8')
df_test = pd.read_csv('test.csv', header=0, index_col='_id', encoding='utf-8')
df_train.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,target
_id,,,,,,,,,,,,,,,,,,,,,
df7489733b004bbe40d3d37b34f82419,54,technician,married,professional.course,unknown,no,no,cellular,aug,tue,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
905a0b9a5456ee962223033473666be3,36,services,single,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
02d69c07ec6d68e31d641edd45ce61cd,53,blue-collar,married,basic.4y,no,yes,no,cellular,aug,mon,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,0
e492fbe0cccaf67cdb5c0944dbc8a167,34,blue-collar,married,basic.9y,no,yes,yes,telephone,may,wed,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
ac2dc6f5835bdea8e71dd99b65208d07,29,services,single,university.degree,no,yes,no,telephone,aug,fri,...,1,999,0,nonexistent,-1.7,94.027,-38.3,0.890,4991.6,0


In [3]:
# посмотрим на данные: структура, пропуски и прочее
df_train.info()
df_train.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 24712 entries, df7489733b004bbe40d3d37b34f82419 to 46d0d25dfd1def79632dd437261d0b5c
Data columns (total 21 columns):
age               24712 non-null int64
job               24712 non-null object
marital           24712 non-null object
education         24712 non-null object
default           24712 non-null object
housing           24712 non-null object
loan              24712 non-null object
contact           24712 non-null object
month             24712 non-null object
day_of_week       24712 non-null object
duration          24712 non-null int64
campaign          24712 non-null int64
pdays             24712 non-null int64
previous          24712 non-null int64
poutcome          24712 non-null object
emp.var.rate      24712 non-null float64
cons.price.idx    24712 non-null float64
cons.conf.idx     24712 non-null float64
euribor3m         24712 non-null float64
nr.employed       24712 non-null float64
target            24712 non-null int64

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,target
count,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000
mean,40.030188,258.345298,2.580163,963.558797,0.171900,0.082713,93.576386,-40.516077,3.619397,5166.988111,0.112779
std,10.462301,261.573344,2.776757,184.224885,0.491511,1.572300,0.579548,4.639359,1.737099,72.516221,0.316329
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000,0.000000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000,0.000000
50%,38.000000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000,0.000000
75%,47.000000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000,0.000000
max,98.000000,4918.000000,43.000000,999.000000,6.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000,1.000000


In [4]:
# взглянем на категориальные переменные
for i in df_train.columns:
    if str(df_train[i].dtype) == 'object':
        print('='*10)
        print(i, ':')
        print(set(df_test[i]))

job :
{'student', 'housemaid', 'self-employed', 'blue-collar', 'management', 'technician', 'unemployed', 'entrepreneur', 'unknown', 'services', 'admin.', 'retired'}
marital :
{'unknown', 'single', 'divorced', 'married'}
education :
{'illiterate', 'professional.course', 'unknown', 'basic.9y', 'university.degree', 'basic.4y', 'high.school', 'basic.6y'}
default :
{'no', 'yes', 'unknown'}
housing :
{'no', 'yes', 'unknown'}
loan :
{'no', 'yes', 'unknown'}
contact :
{'telephone', 'cellular'}
month :
{'apr', 'nov', 'oct', 'dec', 'jul', 'sep', 'jun', 'may', 'aug', 'mar'}
day_of_week :
{'thu', 'tue', 'mon', 'wed', 'fri'}
poutcome :
{'success', 'nonexistent', 'failure'}


In [88]:
# функция предобработки данных
def preproc_data(df_input):
    df_output = df_input.copy()
           
    df_output = df_output.drop(df_output[df_output['duration']==0].index, axis=0)

    n = 10
    
    df_output['age'] = df_output['age'] / n
    df_output['duration'] = df_output['duration'] / n
    df_output['campaign'] = df_output['campaign'] / n
    df_output['pdays'] = df_output['pdays'] / n
    df_output['previous'] = df_output['previous'] / n
    
    df_output['emp.var.rate'] = df_output['emp.var.rate'] / n
    df_output['cons.price.idx'] = df_output['cons.price.idx'] / n
    df_output['cons.conf.idx'] =  df_output['cons.conf.idx'] / n
    df_output['euribor3m'] =  df_output['euribor3m'] / n
    df_output['nr.employed'] =  df_output['nr.employed'] / n
    
    categorical_features = list()
    for i in df_output.columns:
        if str(df_output[i].dtype) == 'object':
            categorical_features.append(i)

   
    df_output = pd.get_dummies(df_output, prefix=categorical_features,
                               columns=categorical_features)
    
    return df_output

In [89]:
# разобьем размеченные данные на обучающую и тестовую выборки
# настроим параметры кросс-валидации
# для воспроизведения результата здесь и далее будем использовать random_state=42
df_train_preproc = df_train.pipe(preproc_data)
df_test_preproc = df_test.pipe(preproc_data)
X, X_test, y, y_test = train_test_split(df_train_preproc.drop('target', axis=1), df_train_preproc.loc[:, 'target'], test_size=0.3, random_state=42)
cv = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

In [90]:
# обучаем дерево решений
# подбираем гиперпараметры
# оцениваем качество на кросс-валидации и тестовой выборке
dt = DecisionTreeClassifier(random_state=42)

dt_pipe = Pipeline([
    ('model', dt)
])

dt_param_grid = {
    'model__criterion': ['gini', 'entropy'],
    'model__max_depth': randint(2, 10),
    'model__min_samples_split': randint(2, 100),
    'model__min_samples_leaf': randint(1, 100),
    'model__class_weight': [None, 'balanced'],
    'model__presort': [False, True]}

dt_model = RandomizedSearchCV(estimator=dt_pipe, param_distributions=dt_param_grid, n_iter=1000, n_jobs=-1,
                                          cv=cv, scoring='roc_auc', random_state=42, verbose=1)

dt_model.fit(X, y)

print(dt_model.best_params_)
print(dt_model.best_score_)

dt_roc_auc = roc_auc_score(y_test, dt_model.predict_proba(X_test)[:, 1])
print(dt_roc_auc)

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   53.7s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed:  3.2min finished


{'model__class_weight': None, 'model__criterion': 'entropy', 'model__max_depth': 7, 'model__min_samples_leaf': 43, 'model__min_samples_split': 22, 'model__presort': False}
0.9370773977859419
0.9381827930158235


In [87]:
# обучаем алгоритм ближайших соседей
# подбираем гиперпараметры
# оцениваем качество на кросс-валидации и тестовой выборке
scaler = MinMaxScaler()
knn = KNeighborsClassifier()

knn_pipe = Pipeline([
    ('scaler', scaler),
    ('model', knn)
])

knn_param_grid = {
    'model__n_neighbors': randint(5, 20),
    'model__weights': ['uniform', 'distance'],
    'model__metric': ['euclidean', 'manhattan']}

knn_model = RandomizedSearchCV(estimator=knn_pipe, param_distributions=knn_param_grid, n_iter=10, n_jobs=-1,
                                           cv=cv, scoring='roc_auc', random_state=42, verbose=1)

knn_model.fit(X, y)

print(knn_model.best_params_)
print(knn_model.best_score_)

knn_roc_auc = roc_auc_score(y_test, knn_model.predict_proba(X_test)[:, 1])
print(knn_roc_auc)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  4.7min finished


{'model__metric': 'manhattan', 'model__n_neighbors': 16, 'model__weights': 'uniform'}
0.8099828596317831
0.8235180340653561


In [91]:
# обучаем логистическую регрессию
# подбираем гиперпараметры
# оцениваем качество на кросс-валидации и тестовой выборке
scaler = MinMaxScaler()
lr = LogisticRegression(random_state=42)

lr_pipe = Pipeline([
    ('scaler', scaler),
    ('model', lr)
])

lr_param_grid = {
    'model__penalty': ['l1', 'l2'],
    'model__C': np.logspace(-3, 3, 250),
    'model__class_weight': [None, 'balanced']}

lr_model = GridSearchCV(estimator=lr_pipe, param_grid=lr_param_grid, n_jobs=-1,
                                  cv=cv, scoring='roc_auc', verbose=1)

lr_model.fit(X, y)

print(lr_model.best_params_)
print(lr_model.best_score_)

lr_roc_auc = roc_auc_score(y_test, lr_model.predict_proba(X_test)[:, 1])
print(lr_roc_auc)

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   35.1s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed: 23.6min finished


{'model__C': 0.623994444168982, 'model__class_weight': 'balanced', 'model__penalty': 'l1'}
0.937014554855388
0.9323079631570944


In [40]:
# лучшее качество показывает модель на основе дерева решений
# предскажем вероятности для неразмеченных данных
# экспортируем результат для дальнейшей оценки на kaggle
df_submission = pd.DataFrame()
df_submission.loc[:,'_id'] = df_test_preproc.reset_index()['_id']
df_submission.loc[:,'target'] = dt_model.predict_proba(df_test_preproc)[:, 1]
df_submission.to_csv('submission.csv', sep=',', index=False, encoding='utf-8')

In [93]:
# попробуем добавить результаты предсказания дерева в лог регрессию
X_, X_test_ = X.copy(), X_test.copy()
X_.loc[:, 'score'] = dt_model.predict_proba(X)[:, 1]
X_test_.loc[:, 'score'] = dt_model.predict_proba(X_test)[:, 1]

# построим новую модель лог регрессии
scaler_ = MinMaxScaler()
lr_ = LogisticRegression(random_state=42)

lr_pipe_ = Pipeline([
    ('scaler', scaler_),
    ('model', lr_)
])

lr_param_grid_ = {
    'model__penalty': ['l1', 'l2'],
    'model__C': np.logspace(-3, 3, 250),
    'model__class_weight': [None, 'balanced']}

lr_model_ = GridSearchCV(estimator=lr_pipe_, param_grid=lr_param_grid_, n_jobs=-1,
                                  cv=cv, scoring='roc_auc', verbose=1)

lr_model_.fit(X_, y)

print(lr_model_.best_params_)
print(lr_model_.best_score_)

lr_roc_auc_ = roc_auc_score(y_test, lr_model_.predict_proba(X_test_)[:, 1])
print(lr_roc_auc_)

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   48.3s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 16.5min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed: 23.4min finished


{'model__C': 0.07577362638765361, 'model__class_weight': 'balanced', 'model__penalty': 'l1'}
0.9493880967515045


ValueError: operands could not be broadcast together with shapes (7413,64) (63,) (7413,64) 

In [95]:
# экспортируем результаты предсказания новой модели лог регресии ( на основе предсказий из модели дерева )
df_test_preproc_ = df_test_preproc.copy()
df_test_preproc_.loc[:, 'score'] = dt_model.predict_proba(df_test_preproc)[:, 1]

df_submission_ = pd.DataFrame()
df_submission_.loc[:,'_id'] = df_test_preproc_.reset_index()['_id']
df_submission_.loc[:,'target'] = lr_model_.predict_proba(df_test_preproc_)[:, 1]
df_submission_.to_csv('submission_.csv', sep=',', index=False, encoding='utf-8')